# 07 - Pandas and Multiple Tables

In [ ]:
# Includes and Standard Magic...
### Standard Magic and startup initializers.

# Load Numpy
import numpy as np
# Load MatPlotLib
import matplotlib
import matplotlib.pyplot as plt
# Load Pandas
import pandas as pd

# This lets us show plots inline and also save PDF plots if we want them
%matplotlib inline
from matplotlib.backends.backend_pdf import PdfPages
matplotlib.style.use('fivethirtyeight')

# These two things are for Pandas, it widens the notebook and lets us display data easily.
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))

# Show a ludicrus number of rows and columns
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

## Importance of using np.nan

Note that we have to import numpy.nan but once we do we can use the groupby and other methods without having to worry about what to do with missing data.

For the example below, what happens when we don't use NaNs in the data table?

In [ ]:
# Make a data frame from a lists
# Try replacing 'XX' with various values..

df = pd.DataFrame({'age':     [12.2, 11.0, 15.6, '--'],
                  'wgt_kg':   [42.3, 40.8, 65.3, 84.2],
                  'hgt_cm':   [145.1, 143.8, 165.3, 185.8],
                  'sex':      ['male', 'female', 'male', 'male'],
                  'country': ['USA', 'AUS', 'EU', 'USA']})
df

In [ ]:
df['age'].describe()

In [ ]:
df.dtypes

In [ ]:
df['age'].sum()

In [ ]:
# Need to make sure it's set as number!
df["age"] = pd.to_numeric(df["age"])

In [ ]:
# But first we have to make sure it's a NAN!!
display(df["age"].replace("--", np.nan))
df["age"].replace("--", np.nan,inplace=True)

In [ ]:
df["age"].sum()

## The Groupby Command

Below we see what happens in Pandas when we use the groupby command.

In [ ]:
df = pd.DataFrame({'age':     [12.2, 11.0, 15.6, 35.1],
                  'wgt_kg':   [42.3, 40.8, 65.3, 84.2],
                  'hgt_cm':   [145.1, 143.8, 165.3, 185.8],
                  'sex':      ['male', 'female', 'male', 'male'],
                  'country': ['USA', 'AUS', 'EU', 'USA']})
df

In [ ]:
df.groupby(['sex']).describe()

# ?? Try describing it..?

In [ ]:
# We can pass operators to groupby to get better results.
df.groupby(['sex']).sum()

In [ ]:
# Can also group by multiple columns
df.groupby(['sex', 'country']).describe()


In [ ]:
grouped = df.groupby(['sex', 'country'])

To get access to a group that we made, use the [get_group() command](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.core.groupby.GroupBy.get_group.html).

We can also get access to the actual tuples and they're types that are generated by the `groupby` commands

In [ ]:
grouped.groups

In [ ]:
grouped.get_group(('female', 'AUS'))

## Joins in Pandas

Let's see a few [joins in Pandas](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.join.html) and how to execute them.

**Note:** We can also use the [merge command in pandas](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.merge.html) to joins as well.  We'll explore this more fully in the lab.  The key idea is that merge let's us pick columns to do the merge/join while join will always join on the index.

In [ ]:
# Careful here! We have to set the index!

df1 = pd.DataFrame([(1, 'foo', 3), (2, 'bar', 2), (3, 'foo', 4), (4, 'foo', 3)],
                  columns=['ID','A', 'B'])
df1.set_index('ID', inplace=True)
display(df1)

df2 = pd.DataFrame([(1, 1.2), (2, 2.5), (3, 2.3), (5, 8.0)], 
                   columns=['ID','C'])
df2.set_index('ID', inplace=True)
display(df2)

In [ ]:
# Inner join
df1.join(df2, how='inner')

In [ ]:
# Can do inner, outter, left, right.
df1.join(df2, how='right')

If we don't have the same column names we can tell Pandas that we want to join on a specific column.

In [ ]:
df1 = pd.DataFrame([(1, 'foo', 3), (2, 'bar', 2), (3, 'foo', 4), (4, 'foo', 3)],
                  columns=['leftid','A', 'B'])
display(df1)

df2 = pd.DataFrame([(1, 1.2), (2, 2.5), (3, 2.3), (5, 8.0)], 
                   columns=['rightid','C'])
df2.set_index('rightid', inplace=True)
display(df2)

In [ ]:
# Inner join
df1.join(df2, how='inner')
# Is this what we wanted?!?!

In [ ]:
# We could have set the index but we can also tell Pandas to do it for us.
df1.join(df2, how='inner', on='leftid')

## Pandas Merge

Or we can skip all this foolishness by using the [pandas merge](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.merge.html) command...

In [ ]:
# Or we can skip all this foolishness by using the merge command...
df1 = pd.DataFrame([(1, 'foo', 3), (2, 'bar', 2), (3, 'foo', 4), (4, 'foo', 3)],
                  columns=['ID','A', 'B'])
display(df1)

df2 = pd.DataFrame([(1, 1.2), (2, 2.5), (3, 2.3), (5, 8.0)], 
                   columns=['ID','C'])
display(df2)

In [ ]:
df1.merge(df2, left_on="ID", right_on="ID", how='inner')